This ETL workflow reads the different datafiles, transforms the data and loads it into the SQL-database 'SmartMeterData'.

In [12]:
import pandas as pd
from pandas import Series,DataFrame
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import matplotlib.dates as mdates
import seaborn as sns
import datetime
from datetime import datetime
from datetime import timezone
import pandas_profiling
import pyodbc
import warnings
%matplotlib inline

# Extract and transform data
The raw datafiles is first read into a dataframe, followed by a set of transformation actions.
Data files are handled per type of information they contain. 

## Data files 'Verbruikshistoriek_elektriciteit'

In [13]:
# Read the csv data files
df_elek_csv_334 = pd.read_csv('Verbruikshistoriek_elektriciteit_541449206005499334_kwartiertotalen.csv',sep=';')
df_elek_csv_907 = pd.read_csv('Verbruikshistoriek_elektriciteit_541449200004926907_kwartiertotalen.csv',sep=';')
df_elek_csv = pd.concat([df_elek_csv_334,df_elek_csv_907])

#Remove unnecessary columns
del df_elek_csv['Meter']
del df_elek_csv['Metertype']
del df_elek_csv['Validatiestatus']

# Rename the values in the column 'Register'
df_elek_csv.loc[df_elek_csv['Register']=='Afname Nacht','Register'] = 'Afname'
df_elek_csv.loc[df_elek_csv['Register']=='Injectie Nacht','Register'] = 'Injectie'
df_elek_csv.loc[df_elek_csv['Register']=='Afname Dag','Register'] = 'Afname'
df_elek_csv.loc[df_elek_csv['Register']=='Injectie Dag','Register'] = 'Injectie'

df_elek_csv

,Van Datum,Van Tijdstip,Tot Datum,Tot Tijdstip,EAN,Register,Volume,Eenheid
0,01/01/2022,00:00:00,01/01/2022,00:15:00,541449206005499334,Afname,"0,067",kWh
1,01/01/2022,00:00:00,01/01/2022,00:15:00,541449206005499334,Injectie,0,kWh
2,01/01/2022,00:15:00,01/01/2022,00:30:00,541449206005499334,Afname,"0,054",kWh
3,01/01/2022,00:15:00,01/01/2022,00:30:00,541449206005499334,Injectie,0,kWh
4,01/01/2022,00:30:00,01/01/2022,00:45:00,541449206005499334,Afname,"0,066",kWh
...,...,...,...,...,...,...,...,...
34739,30/06/2022,23:15:00,30/06/2022,23:30:00,541449200004926907,Injectie,0,kWh
34740,30/06/2022,23:30:00,30/06/2022,23:45:00,541449200004926907,Afname,"0,112",kWh
34741,30/06/2022,23:30:00,30/06/2022,23:45:00,541449200004926907,Injectie,0,kWh
34742,30/06/2022,23:45:00,01/07/2022,00:00:00,541449200004926907,Afname,"0,14",kWh


## Data files 'Verbruikshistoriek_aardgas'

In [14]:
# Read the csv data files
df_gas_csv_732 = pd.read_csv('Verbruikshistoriek_gas_541449206006696732_uurtotalen.csv',sep=';')
df_gas_csv = df_gas_csv_732

# Remove unnecessary colums
del df_gas_csv['Meter']
del df_gas_csv['Metertype']
del df_gas_csv['Validatiestatus']
del df_gas_csv['Calorische Bovenwaarde']

# Remove unnecessary rows
df_gas_csv = df_gas_csv[df_gas_csv['Eenheid'] !='m³']

# Reset index
df_gas_csv = df_gas_csv.reset_index(drop=True)

df_gas_csv

,Van Datum,Van Tijdstip,Tot Datum,Tot Tijdstip,EAN,Register,Volume,Eenheid
0,01/01/2022,00:00:00,01/01/2022,01:00:00,541449206006696732,Afname,"0,466",kWh
1,01/01/2022,01:00:00,01/01/2022,02:00:00,541449206006696732,Afname,"2,022",kWh
2,01/01/2022,02:00:00,01/01/2022,03:00:00,541449206006696732,Afname,"0,034",kWh
3,01/01/2022,03:00:00,01/01/2022,04:00:00,541449206006696732,Afname,"2,101",kWh
4,01/01/2022,04:00:00,01/01/2022,05:00:00,541449206006696732,Afname,"1,931",kWh
...,...,...,...,...,...,...,...,...
4338,30/06/2022,19:00:00,30/06/2022,20:00:00,541449206006696732,Afname,0,kWh
4339,30/06/2022,20:00:00,30/06/2022,21:00:00,541449206006696732,Afname,0,kWh
4340,30/06/2022,21:00:00,30/06/2022,22:00:00,541449206006696732,Afname,0,kWh
4341,30/06/2022,22:00:00,30/06/2022,23:00:00,541449206006696732,Afname,0,kWh


## Data file 'Temperatuur uurdata'

In [15]:
# Read the csv data file
df_temp_csv = pd.read_csv('Temperatuur uurdata.csv',sep=',')

# Reset index
df_temp_csv.reset_index(inplace=True)

# Rename column headers
df_temp_csv = df_temp_csv.rename(columns = {'index':'Datetime'})
df_temp_csv.rename(columns = {'Tijd,"Leuven.cool R04 Diepenbeek - Temperatuur (°C)"':'Temperatuur (°C)'}, inplace = True)

# Check for incorrect measurement values, based in format of datetime
df_temp_csv['CheckFormat'] = df_temp_csv['Datetime'].str.startswith('20')

# Remove rows with incorrect measurement values
df_temp_csv = df_temp_csv[df_temp_csv['CheckFormat'] !=False]

# Delete column used for checking measurement values
del df_temp_csv['CheckFormat']

df_temp_csv

,Datetime,Temperatuur (°C)
0,2022-01-01 01:55:00,12.9
1,2022-01-01 02:55:00,12.9
2,2022-01-01 03:55:00,12.7
3,2022-01-01 04:55:00,12.7
4,2022-01-01 05:55:00,12.6
...,...,...
3647,2022-06-30 19:55:00,13.8
3648,2022-06-30 20:55:00,13.7
3649,2022-06-30 21:55:00,13.5
3650,2022-06-30 22:55:00,13.3


## Data files 'EAN-codes'

In [16]:
# Read the xlsx data file
df_EAN_xlsx = pd.read_excel('EAN-codes.xlsx',header=0)
df_EAN_xlsx

,EAN-code,Energievector,Straat,Huisnummer,Gemeente
0,541449206005499334,Elektriciteit,Sint-Truidersteenweg,79,Hasselt
1,541449206006696732,Aardgas,Sint-Truidersteenweg,79,Hasselt
2,541449200004926907,Elektriciteit,Lindestraat,43,Alken


# Load data
The different dataframes are loaded into the sql database 'SmartMeterData'.  
Before loading the data, all existing tables are dropped to ensure the database only contains data from the recent data files.  
An overview of the different data files, corresponding dataframes and database tables in which the data is loaded:

| Data file                         | Dataframe    | Table in 'SmartmeterData' database   |
|----------------------------------|------|------|
| Verbruikshistoriek_elektriciteit_541449206005499334_kwartiertotalen | df_elek_csv | Elektriciteit |
| Verbruikshistoriek_elektriciteit_541449200004926907_kwartiertotalen | df_elek_csv | Elektriciteit |
| Verbruikshistoriek_gas_541449206006696732_uurtotalen       | df_gas_csv | Aardgas |
| Temperatuur uurdata      | df_temp_csv | Temperatuur |
| EAN-codes       | df_EAN_xlsx | EANcodes |

In [87]:
# Connect to database 'SmartMeterData'
connStr = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER=P210180\SYNTRA_TNVN;DATABASE=SmartMeterData;Trusted_Connection=yes')
cursor = connStr.cursor()

# Drop all existing tables
cursor.execute("""DROP TABLE IF EXISTS Temperatuur, Elektriciteit, Aardgas, EANcodes""");

# Create and fill table 'Temperatuur'
cursor.execute("""CREATE TABLE Temperatuur(
[Datetime] VARCHAR(50),
[Temperatuur (°C)] VARCHAR(50))""");

for index,row in df_temp_csv.iterrows():
    cursor.execute("INSERT INTO dbo.Temperatuur([Datetime],[Temperatuur (°C)]) values (?,?)",
                   row['Datetime'],row['Temperatuur (°C)']) 
    connStr.commit()

# Create and fill table 'Elektriciteit'
cursor.execute("""CREATE TABLE Elektriciteit(
[VanDatum] VARCHAR(50),
[VanTijdstip] VARCHAR(50),
[TotDatum] VARCHAR(50),
[TotTijdstip] VARCHAR(50),
[EAN] VARCHAR(50),
[Register] VARCHAR(50),
[Volume] VARCHAR(50),
[Eenheid] VARCHAR(50))""");

for index,row in df_elek_csv.iterrows():
    cursor.execute("INSERT INTO dbo.Elektriciteit([VanDatum],[VanTijdstip],[TotDatum],[TotTijdstip],[EAN],[Register],[Volume],[Eenheid]) values (?,?,?,?,?,?,?,?)",
                   row['Van Datum'],row['Van Tijdstip'],
                   row['Tot Datum'],row['Tot Tijdstip'],
                   row['EAN'],row['Register'],
                   row['Volume'],row['Eenheid'],) 
    connStr.commit()

# Create and fill table 'Aardgas'
cursor.execute("""CREATE TABLE Aardgas(
[VanDatum] VARCHAR(50),
[VanTijdstip] VARCHAR(50),
[TotDatum] VARCHAR(50),
[TotTijdstip] VARCHAR(50),
[EAN] VARCHAR(50),
[Register] VARCHAR(50),
[Volume] VARCHAR(50),
[Eenheid] VARCHAR(50))""");

for index,row in df_gas_csv.iterrows():
    cursor.execute("INSERT INTO dbo.Aardgas([VanDatum],[VanTijdstip],[TotDatum],[TotTijdstip],[EAN],[Register],[Volume],[Eenheid]) values (?,?,?,?,?,?,?,?)",
                   row['Van Datum'],row['Van Tijdstip'],
                   row['Tot Datum'],row['Tot Tijdstip'],
                   row['EAN'],row['Register'],
                   row['Volume'],row['Eenheid'],) 
    connStr.commit()

# Create and fill table 'EANcodes'
cursor.execute("""CREATE TABLE EANcodes(
[EAN-code] VARCHAR(50),
[Energievector] VARCHAR(50),
[Straat] VARCHAR(50),
[Huisnummer] VARCHAR(50),
[Gemeente] VARCHAR(50))""");

for index,row in df_EAN_xlsx.iterrows():
    cursor.execute("INSERT INTO dbo.EANcodes([EAN-code],[Energievector],[Straat],[Huisnummer],[Gemeente]) values (?,?,?,?,?)",
                   row['EAN-code'],row['Energievector'],
                   row['Straat'],row['Huisnummer'],
                   row['Gemeente']) 

connStr.commit()
cursor.close()
connStr.close()